In [1]:
#importacion de libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import FeatureHasher

In [2]:
#carga de datasets
#Le especifico el tipo a algunas columnas porque sino me tira un warning, podria especificar el verdadero tipo tambien
todoSet = pd.read_csv("events_up_to_01062018.csv",\
                   dtype = {3: object,5: object,
                    6: object,7: object,
                    8: object,9: object,
                    10: object,11: object,
                    12: object,13: object,
                    14: object,15: object,
                    16: object,17: object,
                    18: object,19: object,
                    20: object,21: object,
                    22: object})
trainSet = pd.read_csv("labels_training_set.csv")
usersJunioSet = pd.read_csv("trocafone_kaggle_test.csv")

In [3]:
todoSet.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
trainSet.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [5]:
usersJunioSet.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [6]:
todoSet.dtypes

timestamp                    object
event                        object
person                       object
url                          object
sku                         float64
model                        object
condition                    object
storage                      object
color                        object
skus                         object
search_term                  object
staticpage                   object
campaign_source              object
search_engine                object
channel                      object
new_vs_returning             object
city                         object
region                       object
country                      object
device_type                  object
screen_resolution            object
operating_system_version     object
browser_version              object
dtype: object

In [3]:
#conversion de Object a datetime
todoSet['timestamp'] = pd.to_datetime(todoSet['timestamp'])

# Feature Engineering

In [4]:
#Me quedo con los usuarios que convirtieron en junio(los de train)
usersTrain = todoSet[todoSet['person'].isin(trainSet['person'])].sort_values(by = 'timestamp')

In [5]:
#Creo una columna con solo fecha sin tiempo
usersTrain['fecha'] = usersTrain['timestamp'].dt.date
usersTrain['meses'] = usersTrain['timestamp'].dt.month


# Features Numericos se realizaran en otro archivo

### Entiendo tiene mas sentido tomar peridos mensuales en lugar de diariamente entonces tendria n columnas de features numericos multiplicado por m donde m=cantidad de periodos tomados en este caso m=5

# Features categoricos

In [6]:
usersTrain['fecha'].head()

1417490    2018-01-01
2020472    2018-01-01
2275734    2018-01-01
2305584    2018-01-01
1753252    2018-01-01
Name: fecha, dtype: object

In [7]:
usersTrain['person'].nunique()

19414

In [8]:
#ARMO EL DICCIONARIO PARA USAR PARA EL TEST CON LAS DIMENSIONES
dict_df_cant_col={}

In [9]:
#GENERO FEATURE SEPARADOS POR MES

def acomodar_dataframe(dataframe):
    dataframe=dataframe.pivot_table(index='person',columns='meses').reset_index()
    #dataframe.replace([np.inf, -np.inf], 0,inplace=True)
    dataframe.fillna(0, inplace=True)
    columns = dataframe.columns.tolist()
    columns_name=[]
    for x in columns:
        columns_name=columns_name+[str(x[0])+'-'+str(x[1])]
    columns_name[0]='person'
    #columns_name[1]='label'
    dataframe.columns=columns_name
    dataframe.columns = dataframe.columns.str.replace('-1', "-Enero")
    dataframe.columns = dataframe.columns.str.replace('-2', "-Febrero")
    dataframe.columns = dataframe.columns.str.replace('-3', "-Marzo")
    dataframe.columns = dataframe.columns.str.replace('-4', "-Abril")
    dataframe.columns = dataframe.columns.str.replace('-5', "-Mayo")
#DROPEANDO COLUMNAS NULAS
    #    dataframe= dataframe.loc[:, (dataframe != 0).any(axis=0)]
    return dataframe


In [10]:
#GENERO EL FEATURE A PARTIR DE UN DF PREPROCESADO
def armar_feature(feature,nombre_columna):
    
    #GENERO PARAMETRO FEATURE HASHING
    col=[]
    columna=nombre_columna
    cant_elem=(feature[columna].unique().size) + 1
    #GENERO NOMBRE DE LAS COLUMNAS
    for x in range(1,cant_elem+1):
        col.append("F13_"+columna+"_fh"+str(x))
    #APLICO FEATURE HASHING Y CONCATENO
    fh = FeatureHasher(n_features=cant_elem, input_type='string')
    sp = fh.fit_transform(feature[columna])
    sp=sp.toarray().astype('int8')
    df = pd.DataFrame(sp, columns=col)
    df.reset_index(drop=True, inplace=True)
    feature.reset_index(drop=True, inplace=True)
    #TODA LA DATA JUNTA
    feature=pd.concat([feature, df], axis=1)    
    
    return [feature,cant_elem]

In [11]:
def armar_feature2(feature,nombre_columna,numero_de_columnas):
    
    #GENERO PARAMETRO FEATURE HASHING
    col=[]
    columna=nombre_columna
    cant_elem=numero_de_columnas
    #GENERO NOMBRE DE LAS COLUMNAS
    for x in range(1,cant_elem+1):
        col.append("F13_"+columna+"_fh"+str(x))
    #APLICO FEATURE HASHING Y CONCATENO
    fh = FeatureHasher(n_features=cant_elem, input_type='string')
    sp = fh.fit_transform(feature[columna])
    sp=sp.toarray().astype('int8')
    df = pd.DataFrame(sp, columns=col)
    df.reset_index(drop=True, inplace=True)
    feature.reset_index(drop=True, inplace=True)
    #TODA LA DATA JUNTA
    feature=pd.concat([feature, df], axis=1)    
    
    return [feature,cant_elem]

### COMENZAMOS CON EL ARMADO DE LOS FEATURES

In [12]:
#13mo feature: colores vistos por el usuario por dia
usersColors = usersTrain[usersTrain['event'] == 'viewed product']\
              [['fecha','meses','person','color']].drop_duplicates()

usersColors=armar_feature(usersColors,'color')
dict_df_cant_col['usersColors']=usersColors[1]
usersColors=acomodar_dataframe(usersColors[0])
usersColors.shape

(18570, 321)

In [13]:
#14mo feature: colores comprados por el usuario por dia
usersColorsConversions = usersTrain[usersTrain['event'] == 'conversion']\
                         [['fecha','meses','person','color']].drop_duplicates()

usersColorsConversions=armar_feature(usersColorsConversions,'color')
dict_df_cant_col['usersColorsConversions']=usersColorsConversions[1]
usersColorsConversions=acomodar_dataframe(usersColorsConversions[0])
usersColorsConversions.shape

(2164, 176)

In [14]:
#15mo feature: colores que llegan al checkout por usuario por dia
usersColorsCheckouts = usersTrain[usersTrain['event'] == 'checkout']\
                         [['fecha','meses','person','color']].drop_duplicates()

usersColorsCheckouts=armar_feature(usersColorsCheckouts,'color')
dict_df_cant_col['usersColorsCheckouts']=usersColorsCheckouts[1]
usersColorsCheckouts=acomodar_dataframe(usersColorsCheckouts[0])
usersColorsCheckouts.shape

(16452, 261)

In [15]:
#16mo feature: condition de modelos con checkouts por usuario por dia
usersConditionCheckouts = usersTrain[usersTrain['event'] == 'checkout']\
                         [['fecha','meses','person','condition']].drop_duplicates()

usersConditionCheckouts=armar_feature(usersConditionCheckouts,'condition')
dict_df_cant_col['usersConditionCheckouts']=usersConditionCheckouts[1]
usersConditionCheckouts=acomodar_dataframe(usersConditionCheckouts[0])
usersConditionCheckouts.shape

(16452, 31)

In [16]:
#17mo feature: condition de modelos con conversiones por usuario por dia
usersConditionConversions = usersTrain[usersTrain['event'] == 'conversion']\
                         [['fecha','meses','person','condition']].drop_duplicates()

usersConditionConversions=armar_feature(usersConditionConversions,'condition')
dict_df_cant_col['usersConditionConversions']=usersConditionConversions[1]
usersConditionConversions=acomodar_dataframe(usersConditionConversions[0])
usersConditionConversions.shape

(2164, 31)

In [17]:
#18mo feature: condition de modelos vistos por usuario por dia
usersConditionViewed = usersTrain[usersTrain['event'] == 'viewed product']\
                         [['fecha','meses','person','condition']].drop_duplicates()

usersConditionViewed=armar_feature(usersConditionViewed,'condition')
dict_df_cant_col['usersConditionViewed']=usersConditionViewed[1]
usersConditionViewed=acomodar_dataframe(usersConditionViewed[0])
usersConditionViewed.shape

(18570, 31)

In [18]:
#19mo feature: modelo visto por usuario por dia 
usersModelosVistos = usersTrain[usersTrain['event'] == 'viewed product']\
                         [['fecha','meses','person','model']].drop_duplicates()

usersModelosVistos=armar_feature(usersModelosVistos,'model')
dict_df_cant_col['usersModelosVistos']=usersModelosVistos[1]
usersModelosVistos=acomodar_dataframe(usersModelosVistos[0])
usersModelosVistos.shape


(18570, 971)

In [19]:
#20mo feature: modelo con checkouts por usuario por dia
usersModelsCheckouts = usersTrain[usersTrain['event'] == 'checkout']\
                         [['fecha','meses','person','model']].drop_duplicates()

usersModelsCheckouts=armar_feature(usersModelsCheckouts,'model')
dict_df_cant_col['usersModelsCheckouts']=usersModelsCheckouts[1]
usersModelsCheckouts=acomodar_dataframe(usersModelsCheckouts[0])
usersModelsCheckouts.shape

(16452, 846)

In [20]:
#21mo feature: modelo con conversiones por usuario por dia
usersModelsConversions = usersTrain[usersTrain['event'] == 'conversion']\
                         [['fecha','meses','person','model']].drop_duplicates()

usersModelsConversions=armar_feature(usersModelsConversions,'model')
dict_df_cant_col['usersModelsConversions']=usersModelsConversions[1]
usersModelsConversions=acomodar_dataframe(usersModelsConversions[0])
usersModelsConversions.shape

(2164, 691)

In [21]:
#22mo feature: skus vistos en cierta busqueda en el site por usuario por dia
usersSkusViewedInSearchedProducts = usersTrain[usersTrain['event'] == 'searched products']\
                 [['fecha','meses','person','skus']].dropna().drop_duplicates()

cant_column=int(round(np.sqrt((usersSkusViewedInSearchedProducts['skus'].unique().size))))
usersSkusViewedInSearchedProducts=armar_feature2(usersSkusViewedInSearchedProducts,'skus',cant_column)
dict_df_cant_col['usersSkusViewedInSearchedProducts']=usersSkusViewedInSearchedProducts[1]
usersSkusViewedInSearchedProducts=acomodar_dataframe(usersSkusViewedInSearchedProducts[0])
usersSkusViewedInSearchedProducts.shape

(5802, 556)

In [22]:
#23mo feature: skus vistos en la home page por user por dia
usersSkusViewedHomePage = usersTrain[usersTrain['event'] == 'generic listing']\
                 [['fecha','meses','person','skus']].dropna().drop_duplicates()

cant_column=int(round(np.sqrt((usersSkusViewedHomePage['skus'].unique().size))))
usersSkusViewedHomePage=armar_feature2(usersSkusViewedHomePage,'skus',cant_column)
dict_df_cant_col['usersSkusViewedHomePage']=usersSkusViewedHomePage[1]
usersSkusViewedHomePage=acomodar_dataframe(usersSkusViewedHomePage[0])
usersSkusViewedHomePage.shape

(12601, 291)

In [23]:
#24mo feature: skus vistos en un listado especifico de una marca por user por dia
usersSkusViewedBrandListing = usersTrain[usersTrain['event'] == 'brand listing']\
                              [['fecha','meses','person','skus']].dropna().drop_duplicates()


cant_column=int(round(np.sqrt((usersSkusViewedBrandListing['skus'].unique().size))))
usersSkusViewedBrandListing=armar_feature2(usersSkusViewedBrandListing,'skus',cant_column)
dict_df_cant_col['usersSkusViewedBrandListing']=usersSkusViewedBrandListing[1]
usersSkusViewedBrandListing=acomodar_dataframe(usersSkusViewedBrandListing[0])
usersSkusViewedBrandListing.shape

(9589, 686)

In [24]:
#25mo feature: search terms por usuario por dia
usersSearchTerms = usersTrain[usersTrain['event'] == 'searched products']\
                 [['fecha','meses','person','search_term']].dropna().drop_duplicates()

cant_column=int(round(np.sqrt((usersSearchTerms['search_term'].unique().size))))
usersSearchTerms=armar_feature2(usersSearchTerms,'search_term',cant_column)
dict_df_cant_col['usersSearchTerms']=usersSearchTerms[1]
usersSearchTerms=acomodar_dataframe(usersSearchTerms[0])
usersSearchTerms.shape


(6273, 401)

In [25]:
#26mo feature: screen resolution por usuario por dia 
usersScreenResolution = usersTrain[usersTrain['event'] == 'visited site']\
                 [['fecha','meses','person','screen_resolution']].dropna().drop_duplicates()

usersScreenResolution=armar_feature(usersScreenResolution,'screen_resolution')
dict_df_cant_col['usersScreenResolution']=usersScreenResolution[1]
usersScreenResolution=acomodar_dataframe(usersScreenResolution[0])
usersScreenResolution.shape

(19126, 1431)

In [26]:
#27mo feature: storage de dispositivos convertidos por usuario por dia
usersStorageConversion = usersTrain[usersTrain['event'] == 'conversion']\
                 [['fecha','meses','person','storage']].dropna().drop_duplicates()

usersStorageConversion=armar_feature(usersStorageConversion,'storage')
dict_df_cant_col['usersStorageConversion']=usersStorageConversion[1]
usersStorageConversion=acomodar_dataframe(usersStorageConversion[0])
usersStorageConversion.shape

(2164, 46)

In [27]:
#28mo feature: campaign source por user por dia
usersCampaignSource =  usersTrain[usersTrain['event'] == 'ad campaign hit']\
                 [['fecha','meses','person','campaign_source']].dropna().drop_duplicates()

usersCampaignSource=armar_feature(usersCampaignSource,'campaign_source')
dict_df_cant_col['usersCampaignSource']=usersCampaignSource[1]
usersCampaignSource=acomodar_dataframe(usersCampaignSource[0])
usersCampaignSource.shape

(15884, 121)

In [28]:
#29mo feature: country access user
usersCountry = usersTrain[usersTrain['event'] == 'visited site']\
                 [['fecha','meses','person','country']].dropna().drop_duplicates()

usersCountry=armar_feature(usersCountry,'country')
dict_df_cant_col['usersCountry']=usersCountry[1]
usersCountry=acomodar_dataframe(usersCountry[0])
usersCountry.shape

(19126, 201)

In [ ]:
#30mo feature: city access user
usersCity = usersTrain[usersTrain['event'] == 'visited site']\
                 [['fecha','meses','person','city']].dropna().drop_duplicates()

usersCity=armar_feature(usersCity,'city')
dict_df_cant_col['usersCity']=usersCity[1]
usersCity=acomodar_dataframe(usersCity[0])
usersCity.shape

In [34]:
#31mo feature: region access user
usersRegionAccess = usersTrain[usersTrain['event'] == 'visited site']\
                 [['fecha','meses','person','region']].dropna().drop_duplicates()


usersRegionAccess=armar_feature(usersRegionAccess,'region')
dict_df_cant_col['usersRegionAccess']=usersRegionAccess[1]
usersRegionAccess=acomodar_dataframe(usersRegionAccess[0])
usersRegionAccess.shape

(19126, 426)

In [35]:
#32mo feature: device type visited site users
usersDevicesTypes = usersTrain[usersTrain['event'] == 'visited site']\
                 [['fecha','meses','person','device_type']].dropna().drop_duplicates()

usersDevicesTypes=armar_feature(usersDevicesTypes,'device_type')
dict_df_cant_col['usersDevicesTypes']=usersDevicesTypes[1]
usersDevicesTypes=acomodar_dataframe(usersDevicesTypes[0])
usersDevicesTypes.shape

(19126, 26)

In [36]:
#33mo feature: device type checkout user
usersDeviceTypeCheckouts = usersTrain[usersTrain['event'] == 'checkout']\
                 [['fecha','meses','person','device_type']].dropna().drop_duplicates()


#usersDeviceTypeCheckouts=armar_feature(usersTrain,usersDeviceTypeCheckouts,'device_type')
#usersDeviceTypeCheckouts=acomodar_dataframe(usersDeviceTypeCheckouts)
#print usersDeviceTypeCheckouts.shape
#usersDeviceTypeCheckouts['person'].nunique()
usersDeviceTypeCheckouts.head()


,fecha,meses,person,device_type


In [37]:
#34mo feature: device type conversion user
usersDeviceTypeConversion = usersTrain[usersTrain['event'] == 'conversion']\
                 [['fecha','meses','person','device_type']].dropna().drop_duplicates()

#usersDeviceTypeConversion=armar_feature(usersTrain,usersDeviceTypeConversion,'device_type')
#usersDeviceTypeConversion=acomodar_dataframe(usersDeviceTypeConversion)
#print usersDeviceTypeConversion.shape
#usersDeviceTypeConversion['person'].nunique()

usersDeviceTypeConversion.head()



,fecha,meses,person,device_type


### COMENZAMOS CON EL JOIN DE LOS FEATURES CATEGORICOS
### PRIMERO GENERAMOS EL DF DONDE REALIZAREMOS EL MERGE

In [38]:
#GENERO EL DF CON PERSON Y LABEL PARA EL MERGE
join_features_categoricos=trainSet.loc[:,['person','label']].drop_duplicates()
join_features_categoricos.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [39]:
#GENERO UNA LISTA DE DF PARA REALIZAR EL MERGE
features_categoricos=[usersColors,usersColorsConversions,usersColorsCheckouts,usersConditionCheckouts,usersConditionConversions,usersConditionViewed,usersModelosVistos,usersModelsCheckouts,usersModelsConversions,usersSkusViewedInSearchedProducts,usersSkusViewedHomePage,usersSkusViewedBrandListing,usersSearchTerms,usersScreenResolution,usersStorageConversion,usersCampaignSource,usersCountry,usersCity,usersRegionAccess,usersDevicesTypes]
print ("Cantidad de features categoricos",len(features_categoricos))#32-13+1=20

('Cantidad de features categoricos', 20)


In [40]:
#HAGO EL MERGE CON EL DF CREADO CON PERSON Y LABEL
for feature_to_merge in features_categoricos:
    join_features_categoricos=pd.merge(join_features_categoricos,feature_to_merge, on=['person'],how='left')

In [41]:
#QUITO LOS NA Y COLOCO 0 Y RENOMBRO LAS COLUMNAS Y GUARDO EL ARCHIVO
print join_features_categoricos.shape
join_features_categoricos=join_features_categoricos.fillna(0)
join_features_categoricos.columns = list (join_features_categoricos.columns.str.replace('_x',''))
join_features_categoricos.columns = list (join_features_categoricos.columns.str.replace('_y',''))
join_features_categoricos.to_csv('join_features_categoricos_train_final.csv')
join_features_categoricos.head()

(19414, 16802)


,person,label,F13_color_fh1-Enero,F13_color_fh1-Febrero,F13_color_fh1-Marzo,F13_color_fh1-Abril,F13_color_fh1-Mayo,F13_color_fh10-Enero,F13_color_fh10-Febrero,F13_color_fh10-Marzo,...,F13_device_type_fh4-Enero,F13_device_type_fh4-Febrero,F13_device_type_fh4-Marzo,F13_device_type_fh4-Abril,F13_device_type_fh4-Mayo,F13_device_type_fh5-Enero,F13_device_type_fh5-Febrero,F13_device_type_fh5-Marzo,F13_device_type_fh5-Abril,F13_device_type_fh5-Mayo
0,0566e9c1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
1,6ec7ee77,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
2,abe7a2fb,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
3,34728364,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
4,87ed62de,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [42]:
#GENERO UN DF CON EL DICCIONARIO PARA PASARLE LOS PARAMETROS A TEST
#GUARDO EL DF QUE CONTIENE EN DICCIONARIO
df_with_dict_df_cant_col=pd.DataFrame(dict_df_cant_col.items())
df_with_dict_df_cant_col.columns=['feature','columns']
df_with_dict_df_cant_col.to_csv('dict_with_feature_and_number_of_columns.csv')
df_with_dict_df_cant_col.head()

,feature,columns
0,usersColorsConversions,35
1,usersCountry,40
2,usersRegionAccess,85
3,usersSkusViewedBrandListing,137
4,usersConditionViewed,6
